In [ ]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 11.3 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
!pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.8.10 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("ruslanmv/ai-medical-chatbot", split="train")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

In [ ]:
dataset[10015:10016]

{'Description': ['Q. Do vitamin B12 shots cause derealization symptoms?'],
 'Patient': ['Hi doctor, I have been suffering from the derealization symptoms for the past 5 years. I believe they were caused by B12 shots I was taking to treat autism. After my brain scan I got to know that my inhibition was too high. I do not find any safety net to keep my mind from deconstructing things. A\xa0part of me have a dislike for anything involving neuroplasticity. There are too much dichromatic thinking and lack of magical thinking. Currently, I am on Adderall and Prozac.'],
 'Doctor': ['Hi. I read your query and understand your concerns. I am sorry to know that you are suffering with multiple problems related to thinking and inhibition. Before saying anything I must assure you that vitamin B12 is not known to cause any of the mentioned symptoms and we need to find out the alternative reason for causation of these symptoms. They can be due to autism itself or may be due to other psychopathology th

In [ ]:
subset=dataset[:10000]

In [ ]:
r1_prompt = """You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<input>
 QUESTION:
 {}

 CONTEXT:
 {}

</input>

 ANSWER:
 {}
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
  Questions = examples["Description"]
  Context = examples["Patient"]
  Answers = examples["Doctor"]
  texts = []

  for question, retreived_doc, answer in zip(Questions, Context, Answers):
    text = r1_prompt.format(question, retreived_doc, answer)+EOS_TOKEN
    texts.append(text)

  return {"text": texts}

from datasets import Dataset

subset = Dataset.from_dict(subset)

subset = subset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = subset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 50,
        max_steps = 1000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 20,
        save_steps = 300,
        save_total_limit = 2,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 2 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
20,3.083300
40,2.344900
60,1.982700
80,1.505000
100,1.526900
120,1.281100
140,1.491100
160,1.405200
180,1.234400
200,1.254800


In [ ]:
!zip -r llama_finetune_outputs.zip /content/outputs
from google.colab import files
files.download("llama_finetune_outputs.zip")

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/checkpoint-1000/ (stored 0%)
  adding: content/outputs/checkpoint-1000/tokenizer.json (deflated 85%)
  adding: content/outputs/checkpoint-1000/training_args.bin (deflated 53%)
  adding: content/outputs/checkpoint-1000/rng_state.pth (deflated 26%)
  adding: content/outputs/checkpoint-1000/adapter_model.safetensors (deflated 7%)
  adding: content/outputs/checkpoint-1000/trainer_state.json (deflated 76%)
  adding: content/outputs/checkpoint-1000/README.md (deflated 65%)
  adding: content/outputs/checkpoint-1000/chat_template.jinja (deflated 71%)
  adding: content/outputs/checkpoint-1000/special_tokens_map.json (deflated 71%)
  adding: content/outputs/checkpoint-1000/adapter_config.json (deflated 57%)
  adding: content/outputs/checkpoint-1000/scaler.pt (deflated 64%)
  adding: content/outputs/checkpoint-1000/scheduler.pt (deflated 62%)
  adding: content/outputs/checkpoint-1000/optimizer.pt (deflated 11%)
  adding: content/out

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=subset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        warmup_steps=10,
        max_steps=2050,
        learning_rate=1e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=20,
        # save_steps=300,
        save_total_limit=2,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="/content/drive/MyDrive/llama_models/gguf",
        report_to="none",
    )
)


trainer_stats =trainer.train(resume_from_checkpoint="/content/drive/MyDrive/llama_models/content/outputs/checkpoint-2000")


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/10000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 4 | Total steps = 2,050
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)
	save_steps: 500 (from args) != 300 (from trainer_state.json)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
2020,1.354900
2040,1.210400


In [ ]:
trainer.save_model("/content/drive/MyDrive/llama_models/content/outputs/final_model")
tokenizer.save_pretrained("/content/drive/MyDrive/llama_models/content/outputs/final_model")

('/content/drive/MyDrive/llama_models/content/outputs/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/llama_models/content/outputs/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/llama_models/content/outputs/final_model/chat_template.jinja',
 '/content/drive/MyDrive/llama_models/content/outputs/final_model/tokenizer.json')

In [ ]:
!ls "/content/drive/MyDrive/llama_models/content/outputs/final_model"

adapter_config.json	   README.md		    tokenizer.json
adapter_model.safetensors  special_tokens_map.json  training_args.bin
chat_template.jinja	   tokenizer_config.json


In [ ]:
dataset[100012:100013]

{'Description': ['What causes yellow bruise on the arm with pain?'],
 'Patient': ['Hi, A Few days ago I found my upper left arm very painful, very randomly without injuring myself, or even banging my arm. The next day I discovered yellow bruising which has slowly turned into a rather large red/purple mark which looks like blood surfacing to the skin. This has now spread therefore is now a little bigger. This worries me as it has come out of nowhere'],
 'Doctor': ["Hello, Your symptoms are suggestive of a hematoma, which could be related to a ruptured artery or local inflammation. These are the normal stages after the hematoma and I don't think that you should worry about it. If the problem repeats, I recommend consulting with your attending physician and performing coagulation tests. Hope I have answered your query. Let me know if I can assist you further. Take care Regards, Dr Ilir Sharka, Cardiologist"]}

In [ ]:
from unsloth.chat_templates import get_chat_template
sys_prompt = """You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<input>
QUESTION:
{}

CONTEXT:
{}

</input>
ANSWER:
"""
question= "What causes yellow bruise on the arm with pain?"
context= "Hi, A Few days ago I found my upper left arm very painful, very randomly without injuring myself, or even banging my arm. The next day I discovered yellow bruising which has slowly turned into a rather large red/purple mark which looks like blood surfacing to the skin. This has now spread therefore is now a little bigger. This worries me as it has come out of nowhere"

message = sys_prompt.format(question, context, "")
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": message},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)

In [ ]:
print(response[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

You are a reflective assistant engaging in thorough, iterative reasoning, mimicking human stream-of-consciousness thinking. Your approach emphasizes exploration, self-doubt, and continuous refinement before coming up with an answer.
<input>
QUESTION:
What causes yellow bruise on the arm with pain?

CONTEXT:
Hi, A Few days ago I found my upper left arm very painful, very randomly without injuring myself, or even banging my arm. The next day I discovered yellow bruising which has slowly turned into a rather large red/purple mark which looks like blood surfacing to the skin. This has now spread therefore is now a little bigger. This worries me as it has come out of nowhere

</input>
ANSWER:
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hello. This may not be related to a car accident or anything that cou

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/llama_models/content/outputs/final_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.8.10 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
model.save_pretrained_gguf(
    "llama_finetuned",
    tokenizer,
    quantization_method="q4_k_m"
)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 1.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.37 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:01<00:00, 14.36it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving llama_finetuned/pytorch_model.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at llama_finetuned into f16 GGUF format.
The output location will be /content/llama_finetuned/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: llama_finetuned
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model.bin'
INFO:hf-to-gg

In [ ]:
import shutil
from google.colab import files

shutil.make_archive("llama_finetuned", 'zip', "/content/llama_finetuned")
files.download("llama_finetuned.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>